In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import os
import math
from ast import literal_eval
from collections import defaultdict
from matplotlib_venn import venn3, venn3_circles
from matplotlib_venn import venn2, venn2_circles


# Datasets

## Genres and Platforms

In [ ]:
df_genres = pd.read_csv('../genres.csv', header=None)
df_genres.columns = ['genre_id', 'genre_name']

df_platforms = pd.read_csv('../platforms.csv', header=None)
df_platforms.columns = ['platform_id', 'platform_name']

dict_platform_nintendo = {
    137:"New Nintendo 3DS",
    37:"Nintendo 3DS",
    4:"Nintendo 64",
    416:"Nintendo 64DD",
    20:"Nintendo DS",
    159:"Nintendo DSi",
    18:"Nintendo Entertainment System",
    21:"Nintendo GameCube",
    131:"Nintendo PlayStation",
    130:"Nintendo Switch",
    19:"Super Nintendo Entertainment System",
    5:"Wii",
    41:"Wii U"
}

dict_platform_playstation = {
    38:"PlayStation Portable",
    9:"PlayStation 3",
    131:"Nintendo PlayStation",
    46:"PlayStation Vita",
    48:"PlayStation 4",
    167:"PlayStation 5",
    7:"PlayStation",
    390:"PlayStation VR2",
    165:"PlayStation VR",
    8:"PlayStation 2"
}
        
dict_platform_xbox = {
    11:"Xbox",
    12:"Xbox 360",
    49:"Xbox One",
    169:"Xbox Series X|S"
}

dict_platform_pc = {
    6 : "PC (Microsoft Windows)",
    3: "Linux",
    163: "SteamVR",
    162: "Oculus VR",
    13: "DOS",
    161: "Windows Mixed Reality",
    14: "Mac",
    16: "Amiga",
    386: "Meta Quest 2",
    27: "MSX"
}

dict_platform = {
    **dict_platform_nintendo, 
    **dict_platform_playstation, 
    **dict_platform_xbox, 
    **dict_platform_pc
}

# show genres columns
df_genres.head()

# list all platforms that are not in dict_platform_xbox, dict_platform_nintendo, dict_platform_playstation
df_platforms_pc = df_platforms[~df_platforms['platform_id'].isin(dict_platform_xbox.keys()) & ~df_platforms['platform_id'].isin(dict_platform_nintendo.keys()) & ~df_platforms['platform_id'].isin(dict_platform_playstation.keys())]

# put the df_platforms_pc at dict_platform_pc
dict_platform_pc = dict(zip(df_platforms_pc['platform_id'], df_platforms_pc['platform_name']))

dict_platform


## IGDB + HLTB - Merged

In [ ]:
if not os.path.exists('dataset_merged.csv'):

    PATH = '../igdb_scarper/IGDB_metadata_merged/'

    dataset = pd.DataFrame()
    for platform in os.listdir(PATH):
        # print(platform)

        df_platform = pd.DataFrame()
        
        for file in os.listdir(PATH + platform):

            try:
                df_platform = pd.read_csv(PATH + platform + '/' + file, lineterminator='\n', encoding="utf-8")
                dataset = pd.concat([dataset, df_platform])
            except pd.errors.EmptyDataError:
                # print(file, " is empty and has been skipped.")    
                pass

    dataset['first_release_date'] = pd.to_datetime(dataset['first_release_date'], unit='s')

    # remove the duplicated ids
    dataset = dataset.drop_duplicates(subset=['id'], keep='first')    
    
    dataset.to_csv('dataset_merged.csv', index=False, encoding='utf-8', lineterminator='\n')


dataset = pd.read_csv('dataset_merged.csv', lineterminator='\n', encoding="utf-8",
                    converters={"id": str, 
                                "first_release_date": pd.to_datetime,
                                "platforms": literal_eval,
                                "genres": lambda x: x.strip("[]").split(", ") if x != 'Missing' else 'Missing'
                                })

dataset.info()

In [ ]:
# Total number of games
print("Total number of games: ", dataset.shape[0])


In [ ]:
# show the missing values in the dataset
dataset.isnull().sum()

In [ ]:
# create a csv with the dataset with the platforms that are in dict_platform_xbox
dataset_xbox = dataset[dataset['platforms'].apply(lambda x: any(item in dict_platform_xbox.keys() for item in x))]
dataset_xbox.to_csv('dataset_xbox.csv', index=False, encoding='utf-8', lineterminator='\n')

# create a csv with the dataset with the platforms that are in dict_platform_nintendo
dataset_nintendo = dataset[dataset['platforms'].apply(lambda x: any(item in dict_platform_nintendo.keys() for item in x))]
dataset_nintendo.to_csv('dataset_nintendo.csv', index=False, encoding='utf-8', lineterminator='\n')

# create a csv with the dataset with the platforms that are in dict_platform_playstation
dataset_playstation = dataset[dataset['platforms'].apply(lambda x: any(item in dict_platform_playstation.keys() for item in x))]
dataset_playstation.to_csv('dataset_playstation.csv', index=False, encoding='utf-8', lineterminator='\n')

# create a csv with the dataset with the platforms that are in dict_platform_pc
dataset_pc = dataset[dataset['platforms'].apply(lambda x: any(item in dict_platform_pc.keys() for item in x))]
dataset_pc.to_csv('dataset_pc.csv', index=False, encoding='utf-8', lineterminator='\n')


In [ ]:
# count each platform
print("Number of games in the xbox dataset: ", dataset_xbox.shape[0])
print("Number of games in the nintendo dataset: ", dataset_nintendo.shape[0])
print("Number of games in the playstation dataset: ", dataset_playstation.shape[0])
print("Number of games in the pc dataset: ", dataset_pc.shape[0])

# count the entries of all these datasets together
print("Total number of games in the datasets: ", dataset_xbox.shape[0] + dataset_nintendo.shape[0] + dataset_playstation.shape[0] + dataset_pc.shape[0])

# Platform

## Games per platform

In [ ]:
# number of games per platform
games_per_platform = dataset['platforms'].apply(lambda x: x[0]).value_counts().sort_values(ascending=False)

games_per_platform = pd.DataFrame(games_per_platform).reset_index().rename(columns={'platforms': 'platform_id'})
games_per_platform['platform_id'] = games_per_platform['platform_id'].astype(int)
games_per_platform = games_per_platform.merge(df_platforms, on='platform_id', how='left')
games_per_platform.sort_values(by='count', ascending=False)

games_per_platform['count'].sum()


In [ ]:
# show the platforms that are in dict_platform
games_per_platform[games_per_platform['platform_id'].isin(dict_platform.keys())].sort_values(by='count', ascending=False)   

# show the platforms that are not in dict_platform
# games_per_platform[~games_per_platform['platform_id'].isin(dict_platform.keys())].sort_values(by='count', ascending=False)

In [ ]:
# show a list of the platforms 
games_per_platform[games_per_platform['platform_id'].isin(dict_platform.keys())]['platform_name'].unique()

## Single platform games (exclusives)

In [ ]:

# only single platform games
dataset[dataset['platforms'].apply(lambda x: len(x) == 1)]

# show only the single platform games in their respective platform
games_single_platform = dataset[dataset['platforms'].apply(lambda x: len(x) == 1)]['platforms'].apply(lambda x: x[0]).value_counts()

games_single_platform = pd.DataFrame(games_single_platform).reset_index().rename(columns={'platforms': 'platform_id'})
games_single_platform['platform_id'] = games_single_platform['platform_id'].astype(int)
games_single_platform = games_single_platform.merge(df_platforms, on='platform_id', how='left')
games_single_platform.sort_values(by='count', ascending=False)

# total number of single platform games
print("Total number of single platform games: ", games_single_platform['count'].sum())

games_single_platform


# Genres

## Games per genre

In [ ]:
# remove values missing values in a column
dataset['genres'].replace('Missing', np.nan, inplace=True)
# replace '' with np.nan
dataset['genres'].replace('', np.nan, inplace=True)

print("Number of games with missing genres: ", dataset['genres'].isnull().sum())

dataset['genres'].sort_values(ascending=True)

In [ ]:
# remove genres values missing values
dataset_no_nas = dataset.dropna(subset=['genres'])
# remove lists with empty strings
dataset_no_nas = dataset_no_nas[dataset_no_nas['genres'].apply(lambda x: x[0] != '')]

games_per_genre = dataset_no_nas['genres'].apply(lambda x: x[0]).value_counts().sort_values(ascending=False)

games_per_genre = pd.DataFrame(games_per_genre).reset_index().rename(columns={'genres': 'genre_id'})
games_per_genre['genre_id'] = games_per_genre['genre_id'].astype(int)
games_per_genre = games_per_genre.merge(df_genres, on='genre_id', how='left')
games_per_genre.sort_values(by='count', ascending=False)

games_per_genre['count'].sum()
games_per_genre


In [ ]:
# remove genres values missing values
dataset_xbox_no_nas = dataset_xbox.dropna(subset=['genres'])
# remove lists with empty strings
dataset_xbox_no_nas = dataset_xbox_no_nas[dataset_xbox_no_nas['genres'].apply(lambda x: x[0] != 'M')]

games_per_genre_xbox = dataset_xbox_no_nas['genres'].apply(lambda x: x[0]).value_counts().sort_values(ascending=False)
games_per_genre_xbox = pd.DataFrame(games_per_genre_xbox).reset_index().rename(columns={'genres': 'genre_id'})
games_per_genre_xbox['genre_id'] = games_per_genre_xbox['genre_id'].astype(int)
games_per_genre_xbox = games_per_genre_xbox.merge(df_genres, on='genre_id', how='left')
games_per_genre_xbox['platform'] = 'Xbox'

games_per_genre_xbox.sort_values(by='count', ascending=False)

# do the same for playstation
# remove genres values missing values
dataset_playstation_no_nas = dataset_playstation.dropna(subset=['genres'])
# remove lists with empty strings
dataset_playstation_no_nas = dataset_playstation_no_nas[dataset_playstation_no_nas['genres'].apply(lambda x: x[0] != 'M')]
games_per_genre_playstation = dataset_playstation_no_nas['genres'].apply(lambda x: x[0]).value_counts().sort_values(ascending=False)
games_per_genre_playstation = pd.DataFrame(games_per_genre_playstation).reset_index().rename(columns={'genres': 'genre_id'})
games_per_genre_playstation['genre_id'] = games_per_genre_playstation['genre_id'].astype(int)
games_per_genre_playstation = games_per_genre_playstation.merge(df_genres, on='genre_id', how='left')
games_per_genre_playstation['platform'] = 'Playstation'
games_per_genre_playstation.sort_values(by='count', ascending=False)

# do the same for nintendo
# remove genres values missing values
dataset_nintendo_no_nas = dataset_nintendo.dropna(subset=['genres'])
# remove lists with empty strings
dataset_nintendo_no_nas = dataset_nintendo_no_nas[dataset_nintendo_no_nas['genres'].apply(lambda x: x[0] != 'M')]
games_per_genre_nintendo = dataset_nintendo_no_nas['genres'].apply(lambda x: x[0]).value_counts().sort_values(ascending=False)
games_per_genre_nintendo = pd.DataFrame(games_per_genre_nintendo).reset_index().rename(columns={'genres': 'genre_id'})
games_per_genre_nintendo['genre_id'] = games_per_genre_nintendo['genre_id'].astype(int)
games_per_genre_nintendo = games_per_genre_nintendo.merge(df_genres, on='genre_id', how='left')
games_per_genre_nintendo['platform'] = 'Nintendo'
games_per_genre_nintendo.sort_values(by='count', ascending=False)

# do the same for pc
# remove genres values missing values
dataset_pc_no_nas = dataset_pc.dropna(subset=['genres'])
# remove lists with empty strings
dataset_pc_no_nas = dataset_pc_no_nas[dataset_pc_no_nas['genres'].apply(lambda x: x[0] != 'M')]
dataset_pc_no_nas = dataset_pc_no_nas[dataset_pc_no_nas['genres'].apply(lambda x: x[0] != '')]
games_per_genre_pc = dataset_pc_no_nas['genres'].apply(lambda x: x[0]).value_counts().sort_values(ascending=False)
games_per_genre_pc = pd.DataFrame(games_per_genre_pc).reset_index().rename(columns={'genres': 'genre_id'})
games_per_genre_pc['genre_id'] = games_per_genre_pc['genre_id'].astype(int)
games_per_genre_pc = games_per_genre_pc.merge(df_genres, on='genre_id', how='left')
games_per_genre_pc['platform'] = 'PC'
games_per_genre_pc.sort_values(by='count', ascending=False)

# merge all the platforms
games_per_genre_all = pd.concat([games_per_genre_xbox, 
                                 games_per_genre_playstation, 
                                 games_per_genre_nintendo, 
                                 games_per_genre_pc])

games_per_genre_all.sort_values(by='platform', ascending=False)

# make the table: index, platform, genre, count
games_per_genre_all = games_per_genre_all.reset_index().drop(columns=['index'])
games_per_genre_all = games_per_genre_all.reset_index()
games_per_genre_all = games_per_genre_all.rename(columns={'index': 'genre_rank'})
games_per_genre_all['genre_rank'] = games_per_genre_all['genre_rank'] + 1
games_per_genre_all = games_per_genre_all[['genre_rank', 'platform', 'genre_name', 'count']]
games_per_genre_all.sort_values(by='count', ascending=False)[:10]

# 

In [ ]:

# plot the percentage of the values in dict_rpg
dict_rpg_percent = defaultdict(int)
for key, value in dict_rpg.items():
    dict_rpg_percent[key] = value / len(list_intersections_name) * 100

dict_rpg_percent

# plot a pie chart with the percentage of the values in dict_rpg
fig = plt.figure(figsize=(10, 10))
plt.pie(dict_rpg_percent.values(), labels=dict_rpg_percent.keys(), autopct='%1.1f%%', startangle=90)
plt.title("Percentage of the genres that appear together with 'Role-playing (RPG)'")
plt.show()


# Year

## Games per Year

In [ ]:

# games_per_year = dataset['first_release_date'].dt.year.value_counts().sort_index().reset_index()
# games_per_year.columns = ['year', 'count']
# games_per_year.sort_values(by='year', ascending=False)

# table with the number of games per year keep years type
dataset['first_release_date'].dt.year.value_counts()

# dataset.head()


In [ ]:
df = dataset
# only games that have a genre contains '12'
# df = dataset_no_nas[dataset_no_nas['genres'].apply(lambda x: '12' in x)]

fig, ax = plt.subplots(figsize=(10, 10))
fig.tight_layout()
# fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.2)

ax.hist(df['first_release_date'], 
        bins=len(df['first_release_date'].dt.year.unique()),
        color='lightgray', 
        edgecolor='black',
        align='left',
        linewidth=1.2)

# y labels rounded to 1K, 2K, 3K, etc
ax.set_yticklabels(['{:,.0f}'.format(x) + 'K' for x in ax.get_yticks()/1000])

# add number of games on top of each bar
# for i in ax.patches:
#     ax.text(i.get_x() + i.get_width() / 2, i.get_height() + 100, str(int(i.get_height())), 
#             fontsize=16, rotation=90, ha='center', va='bottom', color='black')

# add a second y axis with the average main value per year from main_per_year
main_per_year = df[(df['main'] != 0)].groupby(
    df['first_release_date'].dt.year)['main'].mean().reset_index().rename(columns={'main': 'mean_main'})
# add the completionist time per year
main_per_year['mean_completionist'] = df[(df['completionist'] != 0)].groupby(
    df['first_release_date'].dt.year)['completionist'].mean().reset_index()['completionist']
# add the main+extra time per year
main_per_year['mean_main_extra'] = df[(df['extra'] != 0)].groupby(
    dataset['first_release_date'].dt.year)['extra'].mean().reset_index()['extra']
# transform the year to datetime
main_per_year['first_release_date'] = pd.to_datetime(main_per_year['first_release_date'], format='%Y')

ax2 = ax.twinx()
ax2.plot(main_per_year['first_release_date'], main_per_year['mean_main'], 
         color='red', linewidth=3, linestyle='--', label='main', alpha=0.5, marker='o', markersize=10, markerfacecolor='red', markeredgecolor='red', markeredgewidth=2)
ax2.plot(main_per_year['first_release_date'], main_per_year['mean_completionist'], 
         color='blue', linewidth=3, linestyle='--', label='completionist', alpha=0.5, marker='o', markersize=10, markerfacecolor='blue', markeredgecolor='blue', markeredgewidth=2)
ax2.plot(main_per_year['first_release_date'], main_per_year['mean_main_extra'], 
         color='green', linewidth=3, linestyle='--', label='main+extra', alpha=0.5, marker='o', markersize=10, markerfacecolor='green', markeredgecolor='green', markeredgewidth=2)
ax2.set_ylabel('Average time to complete the game (hours)')

# add legend
# ax.legend(loc='upper left')
ax2.legend(loc='upper left', fontsize=16)

# increase the size of the x and y axis labels
ax.tick_params(axis='both', which='major', labelsize=16)
ax2.tick_params(axis='both', which='major', labelsize=16)

# increase the font size of the x and y axis labels
ax.xaxis.label.set_size(18)
ax.yaxis.label.set_size(18)
ax2.yaxis.label.set_size(18)

ax.set_xlabel('Year')
ax.set_ylabel('Number of games')
# ax.set_title('Number of games per year')

# # add horizontal grid lines
ax.grid(axis='y', alpha=0.75)
# add more horizontal grid lines
ax.grid(axis='y', which='major', alpha=0.25)


ax.xaxis.set_major_locator(dates.YearLocator(2))
ax.xaxis.set_major_formatter(dates.DateFormatter('%y'))
# show only not even years
ax.xaxis.set_minor_locator(dates.YearLocator())
ax.xaxis.set_minor_formatter(dates.DateFormatter('%y'))

# plt.style.available
fig.savefig('hist.pdf', format='pdf', bbox_inches="tight", dpi=300)


In [ ]:
# compare the time to complete the RPG games with Shooter games
df_rpg = df[df['genres'].apply(lambda x: '12' in x)]
df_shooter = df[df['genres'].apply(lambda x: '5' in x)]

df_rpg.head()
df_shooter.head()

df_rpg['main'].mean()
# df_shooter['main'].mean()

